In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op
from datetime import datetime

# pip
import numpy as np
import xarray as xr

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.waves import GetDistribution



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('KWAJALEIN')


In [3]:
# --------------------------------------
# load data and set parameters

xds_wvs_pts = db.Load_WAVES_partitions()  # waves partitions data
_, xds_TCs_r1_params = db.Load_TCs_r1()   # TCs historical parameters inside big radius
_, xds_TCs_r2_params = db.Load_TCs_r2()   # TCs historical parameters inside small radius

# wave families sectors
wvs_sectors = [(210, 22.5), (22.5, 135)]

# date limits for TCs removal from waves data, and TC time window (hours)
tc_rm_date1 = '1979-01-01'
tc_rm_date2 = '2015-12-31'
tc_time_window = 12



## Calculate Waves Families

In [4]:
# --------------------------------------
# Calculate wave families from waves partitions data and waves sectors

xds_wvs_fam = GetDistribution(xds_wvs_pts, wvs_sectors)

print(xds_wvs_pts)
print()
print(xds_wvs_fam)


<xarray.Dataset>
Dimensions:  (time: 111040)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 ... 2016-12-31T21:00:00
Data variables:
    hs       (time) float64 3.316 3.457 3.382 3.326 ... 2.079 2.105 2.114 2.195
    phs0     (time) float64 0.3466 0.3613 0.3601 0.3554 ... 0.0607 nan 0.146
    phs1     (time) float64 3.203 3.359 3.293 3.251 ... 2.019 2.036 1.977 2.012
    phs2     (time) float64 0.7837 0.7326 0.6812 0.6071 ... 0.3336 0.4789 0.5762
    phs3     (time) float64 nan nan nan nan nan ... 0.2597 0.2621 0.4329 0.5702
    phs4     (time) float64 nan nan nan nan nan ... 0.2465 0.2567 0.2506 0.2657
    phs5     (time) float64 nan nan nan nan ... 0.09064 0.08828 0.1956 0.08443
    tp       (time) float64 9.957 9.872 9.856 9.804 ... 9.569 9.907 10.4 10.71
    ptp0     (time) float64 3.214 2.404 2.407 10.35 ... 3.881 2.952 nan 4.371
    ptp1     (time) float64 9.892 9.741 9.719 9.626 ... 9.575 9.872 9.398 9.87
    ptp2     (time) float64 12.62 12.57 12.52 12.49 ... 13.63 1

In [5]:
# --------------------------------------
# Remove TCs from WAVES data (partitions and families)

# remove TCs before 1979 and after 2015 (r1)
d1 = np.datetime64(tc_rm_date1)
d2 = np.datetime64(tc_rm_date2)
dmin_dates = xds_TCs_r1_params.dmin_date.values[:]

p1 = np.where(dmin_dates >= d1)[0][0]
p2 = np.where(dmin_dates <= d2)[0][-1]
xds_TCs_r1_params = xds_TCs_r1_params.isel(storm = slice(p1, p2))

# select storms inside big circle but outside small circle
storms_r1 = xds_TCs_r1_params.storm.values[:]
storms_r2 = xds_TCs_r2_params.storm.values[:]  # r2 storms will be numerically solved

storms_sel = [s for s in storms_r1 if not s in storms_r2]


# for each storm: find hs_max instant and clean waves data inside "hs_max window"
xds_wvs_pts_noTCs = xds_wvs_pts.copy()
xds_wvs_fam_noTCs = xds_wvs_fam.copy()

# store removed waves (TCs window) families inside a storm category dictionary
d_wvs_fam_cats = {'{0}'.format(k):[] for k in range(6)}

for s in storms_sel:

    # storm dates at dist_min and storm_end   
    xds_s = xds_TCs_r1_params.sel(storm=s)
 
    date_dmin = xds_s.dmin_date.values
    date_last = xds_s.last_date.values
    cat = xds_s.category.values

    xds_wvs_s = xds_wvs_pts.sel(
        time = slice(str(date_dmin), str(date_last))
    )

    # get hs_max date 
    t_hs_max = xds_wvs_s.where(
        xds_wvs_s.hs ==  xds_wvs_s.hs.max(), drop=True
    ).time.values[:][0]

    # hs_max time window 
    w1 = t_hs_max - np.timedelta64(tc_time_window,'h')
    w2 = t_hs_max + np.timedelta64(tc_time_window,'h')

    # clean waves partitions 
    xds_wvs_pts_noTCs = xds_wvs_pts_noTCs.where(
        (xds_wvs_pts_noTCs.time < w1) |
        (xds_wvs_pts_noTCs.time > w2)
    )
    # clean waves families
    xds_wvs_fam_noTCs = xds_wvs_fam_noTCs.where( 
        (xds_wvs_fam_noTCs.time < w1) |
        (xds_wvs_fam_noTCs.time > w2)
    )
    
    # store waves families inside storm-removal-window
    xds_fams_s = xds_wvs_fam.sel(time = slice(w1, w2))
    d_wvs_fam_cats['{0}'.format(cat)].append(xds_fams_s)
    
# merge removal-window waves families data
for k in d_wvs_fam_cats.keys():
    d_wvs_fam_cats[k] = xr.merge(d_wvs_fam_cats[k])
    
print('\nWaves Partitions (TCs removed) ')
print(xds_wvs_pts_noTCs)
print('\nWaves Families   (TCs removed) ')
print(xds_wvs_fam_noTCs)    
    
# store results
db.Save_WAVES_ptsfams_noTCs(xds_wvs_pts_noTCs, xds_wvs_fam_noTCs)

# store waves families data (TCs removal window)
db.Save_WAVES_fams_TCs_categ(d_wvs_fam_cats)



Waves Partitions (TCs removed) 
<xarray.Dataset>
Dimensions:  (time: 111040)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 ... 2016-12-31T21:00:00
Data variables:
    hs       (time) float64 3.316 3.457 3.382 3.326 ... 2.079 2.105 2.114 2.195
    phs0     (time) float64 0.3466 0.3613 0.3601 0.3554 ... 0.0607 nan 0.146
    phs1     (time) float64 3.203 3.359 3.293 3.251 ... 2.019 2.036 1.977 2.012
    phs2     (time) float64 0.7837 0.7326 0.6812 0.6071 ... 0.3336 0.4789 0.5762
    phs3     (time) float64 nan nan nan nan nan ... 0.2597 0.2621 0.4329 0.5702
    phs4     (time) float64 nan nan nan nan nan ... 0.2465 0.2567 0.2506 0.2657
    phs5     (time) float64 nan nan nan nan ... 0.09064 0.08828 0.1956 0.08443
    tp       (time) float64 9.957 9.872 9.856 9.804 ... 9.569 9.907 10.4 10.71
    ptp0     (time) float64 3.214 2.404 2.407 10.35 ... 3.881 2.952 nan 4.371
    ptp1     (time) float64 9.892 9.741 9.719 9.626 ... 9.575 9.872 9.398 9.87
    ptp2     (time) float64 12